In [1]:
import os

os.environ["CONFIG_APP_DIR"] = "config"


In [2]:
import sys
import os
import pickle
os.chdir("..")
from omegaconf import OmegaConf

import json
from etl_pipeline.data_processor_engine.json_engine.json_engine import JsonProcessingEngine
from etl_pipeline.custom.ms.payload_loader import PayloadLoader

/env/ds/anaconda/envs/pipeline/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
from etl_pipeline.config import pipeline_config
cn = pipeline_config.cn

In [4]:
# alert = load_alert()

In [5]:
# payload = load_alert()


# Definition

In [6]:
from pipelines.ms.ms_pipeline import MSPipeline
from etl_pipeline.config import pipeline_config


In [7]:
with open(f'notebooks/sample/wm_address_in_payload_format.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)
payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [8]:
from etl_pipeline.config import load_agent_configs, pipeline_config

In [9]:
payload = payload_json

In [10]:
# engine = JsonProcessingEngine(pipeline_config)
pipeline = MSPipeline.build_pipeline(MSPipeline)

The following fields have not been found in any file:
['ALL_CONNECTED_PARTY_CITIZENSHIP_COUNTRIES', 'ALL_CONNECTED_TAX_IDS', 'ALL_CONNECTED_PARTY_RESIDENCY_COUNTRIES', 'ALL_CONNECTED_PARTY_BIRTH_COUNTRIES', 'ALL_CONNECTED_GOVT_IDS', 'ALL_CONNECTED_COUNTRY_OF_INCORPORATION']


## transform standardized to cleansed

In [11]:
payload = pipeline.transform_standardized_to_cleansed(payload)

ALL_CONNECTED_ACCOUNT_NAMES# Transform standardized to application

In [12]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

### payload

In [13]:
len(new_payloads)

2

In [14]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
        
    match = payload['watchlistParty']['matchRecords']
    assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
    print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
    try:
        print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
    except:
        break


PAYLOAD
122438658
['04/31/1910', '02/31/1900'] ['MAY 6, 1981']
['UNITED STATES OF AMERICA', 'United States'] ['United States of America', 'US|FL;US']
PAYLOAD
122438659
['04/31/1910', '02/31/1900'] ['MAY 6, 1981']
['UNITED STATES OF AMERICA', 'United States'] ['PL', 'Poland']


In [15]:
import pandas as pd
out_payload = pd.DataFrame([payload['watchlistParty']['matchRecords'] for payload in new_payloads])

In [16]:
out_payload[[col for col in out_payload.columns if "dobs" in col]]

,ap_all_dobs_aggregated,wl_all_dobs_aggregated
0,"[04/31/1910, 02/31/1900]","[MAY 6, 1981]"
1,"[04/31/1910, 02/31/1900]","[MAY 6, 1981]"


In [17]:
import pickle
with open("tests/shared/parsed_payload.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

    

In [18]:
with open(f'notebooks/sample/big_fat_flat_payload.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)
payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [19]:
from etl_pipeline.config import load_agent_configs, pipeline_config

In [20]:
payload = payload_json

## transform standardized to cleansed

In [21]:
payload = pipeline.transform_standardized_to_cleansed(payload)

ALL_CONNECTED_ACCOUNT_NAMES# Transform standardized to application

In [22]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

### payload

In [23]:
len(new_payloads)

9

In [24]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
        
    match = payload['watchlistParty']['matchRecords']
    assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
    print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
    try:
        print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
    except:
        break


PAYLOAD
94590752
[] ['01/29/1971', '1971', '29', '01']
['AHMEDOUA'] ['FRANCE', 'FR']
PAYLOAD
94590752
[] ['07', '1986', '07/27/1986', '27']
['AHMEDOUA'] ['LY', 'LIBYAN ARAB JAMAHIRIYA']
PAYLOAD
94590752
[] ['1977', '07/06/1977', '07', '06']
['AHMEDOUA'] []
PAYLOAD
94590752
[] ['08/17/1938', '17', '08', '1938']
['AHMEDOUA'] ['INDONESIA', 'ID']
PAYLOAD
94590752
[] ['04', '1958', '04/15/1958', '15']
['AHMEDOUA'] ['US', 'UNITED STATES']
PAYLOAD
94590752
[] []
['AHMEDOUA'] ['RU|PR. LENINA, D. 179', 'RUSSIAN FEDERATION']
PAYLOAD
94590752
[] ['01', '01/01/1982', '1982']
['AHMEDOUA'] ['MALI|MALI', 'ML|ML']
PAYLOAD
94590752
[] ['08/27/1981', '08', '27', '1981']
['AHMEDOUA'] []
PAYLOAD
94590752
[] ['08/17/1938', '17', '08', '1938']
['AHMEDOUA'] ['INDONESIA', 'ID']


In [25]:
import pickle
with open("tests/shared/big_fat_payload_parsed.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [26]:
# import pickle
# with open("tests/shared/parsed_payload.pkl", "rb") as f:
#     p = pickle.load(f)

# import pandas as pd
# reference_payload = pd.DataFrame([match for payload in p  for match in payload['watchlistParty']['matchRecords']])



In [27]:
with open(f'notebooks/sample/wm_address_in_payload_format_2_input_3_match_records.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)



payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [28]:
payload = payload_json

In [29]:
payload = pipeline.transform_standardized_to_cleansed(payload)

In [30]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

In [31]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
        
    match = payload['watchlistParty']['matchRecords']
    assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
    print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
    try:
        print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
    except:
        break


PAYLOAD
123
['04/31/1910', '02/31/1900'] ['MAY 6, 1981']
['UNITED STATES OF AMERICA', 'United States'] ['United States of America', 'US|FL;US']
PAYLOAD
122438659
['04/31/1910', '02/31/1900'] ['MAY 6, 1981']
['UNITED STATES OF AMERICA', 'United States'] ['United States of America', 'US|FL;US']
PAYLOAD
122438659
['04/31/1910', '02/31/1900'] ['MAY 6, 1981']
['UNITED STATES OF AMERICA', 'United States'] ['PL', 'Poland']


In [32]:
new_dict = {}
def return_key(dict_, prefix):
    
    if isinstance(dict_, list):
        for num, i in enumerate(dict_):
            return_key(i, prefix +f"[{num}]")
        return
    if isinstance(dict_, str) or dict_ is None:
        new_dict[prefix] = dict_
        return

    for key in dict_:
        if prefix:
            basic_prefix = prefix + "." + key
        else:
            basic_prefix = key
        return_key(dict_[key], basic_prefix)

In [33]:
import pickle
with open("tests/shared/parsed_payload_2_payload.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [34]:
with open(f'notebooks/sample/wm_party_payload_without_supplemental_info.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)



payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [35]:
payload = payload_json

In [36]:
payload = pipeline.transform_standardized_to_cleansed(payload)

In [37]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

In [38]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
        
    match = payload['watchlistParty']['matchRecords']
    assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
    print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
    try:
        print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
    except:
        break


PAYLOAD
122438658
[] ['MAY 6, 1981']
[] ['|FL;US']
PAYLOAD
122438659
[] ['MAY 6, 1981']
[] ['PL', 'Poland']


In [39]:
import pickle
with open("tests/shared/empty_payload.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [40]:
with open(f'notebooks/sample/wm_party_payload_with_partial_supplemental_info.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)



payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [41]:
payload = payload_json

In [42]:
payload = pipeline.transform_standardized_to_cleansed(payload)

No parties
No accounts
No parties
No accounts
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No accounts
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No parties


In [43]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

In [44]:
new_payloads

[{'alertedParty': {'headerInfo': {'currentVersionId': '122438658',
    'datasetId': '1044',
    'datasetName': 'R_US_Active_Party',
    'firstVersionCreatedDt': '2020-01-08T09:31:30.264-05:00',
    'inputVersionSample': 'R_US_Active_Address_A05003324172_2020-01-07-07.06.',
    'lastVersionUpdatedDt': '2020-01-08T09:31:30.264-05:00',
    'masterId': '72951854',
    'masterVersion': '412740c151535f0756e8dbec7440b726c7a3e135',
    'stopDescriptors': {'stopDescriptor': {'name': 'Joe Doe'}},
    'uniqueCustomerId': 'R_US_Active_Address_A05003324172_2020-01-07-07.06.28.836480'},
   'inputRecordHist': {'inputRecords': {'createdDate': '01/08/20',
     'fields': [{'isScreenable': 'false',
       'name': 'SOURCE_REF',
       'sortOrder': '1',
       'value': 'R_US_Active_Address_A05003324172_2020-01-07-07.06.28.836480'},
      {'isScreenable': 'false',
       'name': 'UNIQUE_KEY',
       'sortOrder': '2',
       'value': 'A05003324172'},
      {'isScreenable': 'false', 'name': 'BARCODE', 'sortOr

In [45]:
import pickle
with open("tests/shared/wm_party_payload_with_partial_supplemental_info.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [46]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
        
    match = payload['watchlistParty']['matchRecords']
    assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
    print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
    try:
        print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
    except:
        break


PAYLOAD
122438658
[] ['MAY 6, 1981']
[] ['|FL;US']
PAYLOAD
122438659
[] ['MAY 6, 1981']
[] ['PL', 'Poland']


In [47]:
from etl_pipeline.config import load_agent_config

In [48]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload[[i for i in out_payload.columns  if i.endswith("_aggregated")]]

AttributeError: 'int' object has no attribute 'endswith'

In [49]:
from omegaconf import OmegaConf
alert_agents_config = {
        "alert_type": OmegaConf.load(os.path.join(CONFIG_APP_DIR, "agents", "agents.yaml"))
    }


import omegaconf
def get_fields(dict_):
    fields = []
    for key in dict_:
        
        if isinstance(dict_[key], omegaconf.dictconfig.DictConfig):
            new_fields = get_fields(dict_[key])
            fields.extend(new_fields)
        else:
#             import pdb; pdb.set_trace()
            fields.extend(dict_[key])
    return fields

fields = [i.split('.')[-1] for i in get_fields(alert_agents_config['alert_type']) if "alertedParty.inputRecordHist.inputRecords.INPUT_FIELD" not in i]
    
    

from glob import glob

files = glob("pipelines/**/*.py",  recursive=True)
files.extend(glob("etl_pipeline/**/*.py",  recursive=True))

fields_to_check = list(fields)
for file in files:
    with open(file) as f:
        text = f.read()
    new_fields_to_check = []
    
    for field in fields_to_check:
        if field in text:
            continue
        new_fields_to_check.append(field)
    fields_to_check = new_fields_to_check
        
    

fields_to_check

CONFIG_APP_DIR = os.environ["CONFIG_APP_DIR"] 

NameError: name 'CONFIG_APP_DIR' is not defined

In [ ]:
import pickle

In [ ]:
with open("/tmp/request.pkl", 'rb') as f:
    r = pickle.load(f)

In [ ]:
from silenteight.datasource.api.name.v1.name_pb2 import (
    NameFeatureInput,
)

In [ ]:
from google.protobuf.any_pb2 import Any

In [ ]:
some_any2 = Any()
message = NameFeatureInput()
some_any2.Pack(NameFeatureInput())
# some_any2.CopyFrom(i.feature_inputs[0].agent_feature_input)


In [ ]:
i.feature_inputs[0].agent_feature_input

In [ ]:
i.feature_inputs[0].agent_feature_input.Unpack(message)

In [ ]:
message

In [ ]:
unpacked_msg = extension_pb2.GeneralContent_Extension())


In [ ]:
NameFeatureInput.ParseFromString()

In [ ]:
from sile
i.feature_inputs[0].agent_feature_input.value.decode('utf-8')

In [ ]:
type(i.feature_inputs[0].agent_feature_input.type_url)

In [ ]:
for i in response.agent_inputs:
    print(i.feature_inputs)

In [ ]:
for 